<a href="https://colab.research.google.com/github/sanjanasrinivas22/BIS/blob/main/BisLabExam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import random
import math
from heapq import nsmallest
def random_path(graph, src, dst, max_hops=100):
    path = [src]
    visited = set([src])
    current = src
    hops = 0
    while current != dst and hops < max_hops:
        neighbors = list(graph.get(current, {}).keys())
        if not neighbors:
            break
        next_node = random.choice(neighbors)
        if next_node in visited and next_node != dst:
            choices = [v for v in neighbors if v not in visited or v == dst]
            if not choices:
                next_node = random.choice(neighbors)
            else:
                next_node = random.choice(choices)
        path.append(next_node)
        visited.add(next_node)
        current = next_node
        hops += 1
    if current == dst:
        return path
    return None
def path_cost(graph, path):
    cost = 0.0
    for i in range(len(path) - 1):
        a, b = path[i], path[i + 1]
        w = graph[a].get(b);
        if w is None:
            return float('inf')
        cost += w
    return cost
def perturb_path(graph, path, src, dst, max_hops=100):
    if path is None or len(path) < 3:
        return random_path(graph, src, dst, max_hops)
    n = len(path)
    i = random.randint(0, n - 2)
    j = random.randint(i + 1, n - 1)
    a = path[i]
    b = path[j]
    new_mid = random_path_between(graph, a, b, max_hops=max_hops)
    if new_mid is None:
        return None
    new_path = path[:i] + new_mid + path[j + 1:]
    if new_path[0] != src:
        return None
    if new_path[-1] != dst:
        return None
    seen = set()
    simple = []
    for node in new_path:
        if node in seen:
            continue
        simple.append(node)
        seen.add(node)
    return simple
def random_path_between(graph, a, b, max_hops=100):
    if a == b:
        return [a]
    path = [a]
    visited = set([a])
    current = a
    hops = 0
    while current != b and hops < max_hops:
        neighbors = list(graph.get(current, {}).keys())
        if not neighbors:
            break
        next_node = random.choice(neighbors)
        if next_node in visited and next_node != b:
            choices = [v for v in neighbors if v not in visited or v == b]
            if not choices:
                next_node = random.choice(neighbors)
            else:
                next_node = random.choice(choices)
        path.append(next_node)
        visited.add(next_node)
        current = next_node
        hops += 1
    if current == b:
        return path
    return None
def cuckoo_search_routing(graph, src, dst, n=25, pa=0.25, max_iter=200, max_hops=100):
    nests = []
    for _ in range(n):
        p = None
        for _ in range(10):
            p = random_path(graph, src, dst, max_hops)
            if p:
                break
        nests.append(p)
    fitness = [path_cost(graph, p) if p else float('inf') for p in nests]
    best_idx = min(range(len(fitness)), key=lambda k: fitness[k])
    best_path = nests[best_idx]
    best_cost = fitness[best_idx]
    for t in range(max_iter):
        for i in range(n):
            new_p = perturb_path(graph, nests[i], src, dst, max_hops)
            if new_p is None:
                new_p = random_path(graph, src, dst, max_hops)
            new_f = path_cost(graph, new_p) if new_p else float('inf')
            if new_f < fitness[i]:
                nests[i] = new_p
                fitness[i] = new_f
                if new_f < best_cost:
                    best_cost = new_f
                    best_path = new_p
        k = max(1, int(pa * n))
        worst_idx = nsmallest(k, range(n), key=lambda x: -fitness[x])
        for idx in worst_idx:
            p = None
            for _ in range(10):
                p = random_path(graph, src, dst, max_hops)
                if p:
                    break
            nests[idx] = p
            fitness[idx] = path_cost(graph, p) if p else float('inf')
            if fitness[idx] < best_cost:
                best_cost = fitness[idx]
                best_path = p
    return best_path, best_cost
if __name__ == '__main__':
    graph = {
        'A': {'B': 2, 'C': 5},
        'B': {'A': 2, 'C': 1, 'D': 4},
        'C': {'A': 5, 'B': 1, 'D': 1, 'E': 7},
        'D': {'B': 4, 'C': 1, 'E': 3, 'F': 5},
        'E': {'C': 7, 'D': 3, 'F': 2},
        'F': {'D': 5, 'E': 2}
    }
    src = 'A'
    dst = 'F'
    path, cost = cuckoo_search_routing(graph, src, dst, n=30, pa=0.3, max_iter=300)
    print('best path', path)
    print('cost', cost)


best path ['A', 'B', 'C', 'D', 'F']
cost 9.0
